# Understanding Prompts from Phoenix

In [ ]:
! pip install langgraph-supervisor langchain-openai python-dotenv arize-phoenix-otel openinference-instrumentation-langchain arize-phoenix-client

In [65]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
from phoenix.client import Client
import os

load_dotenv()

OPENAI_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL")
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "http://localhost:6006"

from phoenix.otel import register

# configure the Phoenix tracer
tracer_provider = register(
    project_name="default",  # Default is 'default'
    auto_instrument=True,  # Auto-instrument your app based on installed OI dependencies
)

model = ChatOpenAI(
    model=OPENAI_MODEL,
    api_key=OPENAI_KEY,
)

client = Client()

Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [111]:
prompt = client.prompts.get(prompt_version_id="UHJvbXB0VmVyc2lvbjo5")

In [112]:
print(prompt._dumps())

{'model_provider': 'OPENAI', 'model_name': 'gpt-4o', 'template': {'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': "You are a chatbot and you need to answer questions by user on Areize Phoneix project.\n\nBefore answering the question, you should greet the user. \nHere are a few examples of greeting:\n- Hey, that's a great question\n- Wow, that's a nice one\n\nIMPORTANT: Answer to questions based on the knowledge you have in your context. Do not answer outside that and do not make up things."}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'Phoenix is an open-source observability tool designed for experimentation, evaluation, and troubleshooting of AI and LLM applications. It allows AI engineers and data scientists to quickly visualize their data, evaluate performance, track down issues, and export data to improve.\n\nPhoenix is built by Arize AI, the company behind the industry-leading AI observability platform, and a set of core contributors.\n\n{{questi

In [124]:
formatted_prompt = prompt.format(
    variables={
        "question": "What Arize Phoenix is used for?",
    },
)

In [125]:
# Convert to LangChain messages
messages = []
for msg in formatted_prompt.messages:
    if msg["role"] == "system":
        messages.append(SystemMessage(content=msg["content"]))
    elif msg["role"] == "user":
        messages.append(HumanMessage(content=msg["content"]))

In [126]:
print("Formatted Messages:", messages)

Formatted Messages: [SystemMessage(content="You are a chatbot and you need to answer questions by user on Areize Phoneix project.\n\nBefore answering the question, you should greet the user. \nHere are a few examples of greeting:\n- Hey, that's a great question\n- Wow, that's a nice one\n\nIMPORTANT: Answer to questions based on the knowledge you have in your context. Do not answer outside that and do not make up things.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Phoenix is an open-source observability tool designed for experimentation, evaluation, and troubleshooting of AI and LLM applications. It allows AI engineers and data scientists to quickly visualize their data, evaluate performance, track down issues, and export data to improve.\n\nPhoenix is built by Arize AI, the company behind the industry-leading AI observability platform, and a set of core contributors.\n\nWhat Arize Phoenix is used for?', additional_kwargs={}, response_metadata={})]


In [127]:
response = model.invoke(messages)

In [128]:
print("Response: \n", response.content)

Response: 
 Hey, that's a great question! Arize Phoenix is used for experimentation, evaluation, and troubleshooting of AI and LLM applications. It helps AI engineers and data scientists visualize their data, evaluate performance, identify issues, and export data to improve their models and processes.
